<a href="https://colab.research.google.com/github/shrikrishnachannawar/text_summarizer/blob/main/ai_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers

## 0. Installing transformers and dependancies

In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## 1. Loding summarize pipeline

In [ ]:
summurizer =pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

##2.Getting blog post from the web

In [ ]:
#here you can add the url of any document or blog post you want to summarize
URL="https://science.nasa.gov/astrophysics/focus-areas/black-holes/"

In [ ]:
r=requests.get(URL)

In [ ]:
r.text

'<!DOCTYPE html>\n<html  lang="en">\n<head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n<title>Black Holes - NASA Science</title>\n<link rel="apple-touch-icon" sizes="180x180" href="https://science.nasa.gov/apple-touch-icon.png">\n<link rel="preload" href="https://science.nasa.gov/font-face.css" as="style">\n<link rel="preload" href="https://smd-cms.nasa.gov/wp-content/client-mu-plugins/uswds-framework/uswds/fonts/public-sans/PublicSans-Regular.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://smd-cms.nasa.gov/wp-content/client-mu-plugins/uswds-framework/uswds/fonts-split/inter/UcC73FwrK3iLTeHuS_fvQtMwCp50KnMa1ZL7.woff2" as="font" type="font/woff2" crossorigin="anonymous">\n<link rel="preload" href="https://smd-cms.nasa.gov/wp-content/client-mu-plugins/uswds-framework/uswds/fonts-split/db-mono/aFTU7PB1QTsUX8KYthqQBA.woff2" as="font" type="font/woff2" cros

In [ ]:
soup=BeautifulSoup(r.text,'html.parser')
results=soup.find_all(['h1','h2','p'])

In [ ]:
results

[<h1 class="display-48 grid-col-12 desktop:grid-col-11 margin-y-0">Black Holes</h1>,
 <p class="margin-0 padding-0">Don't let the name fool you: a black hole is anything but empty space. Rather, it is a great amount of matter packed into a very small area - think of a star ten times more massive than the Sun squeezed into a sphere approximately the diameter of New York City. The result is a gravitational field so strong that nothing, not even light, can escape. In recent years, NASA instruments have painted a new picture of these strange objects that are, to many, the most fascinating objects in space.</p>,
 <p class="margin-0 padding-0">Intense X-ray flares thought to be caused by a black hole devouring a star. (Video)</p>,
 <p class="margin-0 padding-0"><a href="http://www.nasa.gov/mission_pages/swift/bursts/devoured-star.html">Watch the Video</a></p>,
 <p class="margin-0 padding-0"><br/>The idea of an object in space so massive and dense that light could not escape it has been aroun

In [ ]:
text=[result.text for result in results]
article=''.join(text)

In [ ]:
article

'Black HolesDon\'t let the name fool you: a black hole is anything but empty space. Rather, it is a great amount of matter packed into a very small area - think of a star ten times more massive than the Sun squeezed into a sphere approximately the diameter of New York City. The result is a gravitational field so strong that nothing, not even light, can escape. In recent years, NASA instruments have painted a new picture of these strange objects that are, to many, the most fascinating objects in space.Intense X-ray flares thought to be caused by a black hole devouring a star. (Video)Watch the VideoThe idea of an object in space so massive and dense that light could not escape it has been around for centuries. Most famously, black holes were predicted by Einstein\'s theory of general relativity, which showed that when a massive star dies, it leaves behind a small, dense remnant core. If the core\'s mass is more than about three times the mass of the Sun, the equations showed, the force o

## 3. Chunk Text

In [ ]:
article=article.replace('.','.<eos>')
article=article.replace('!','!<eos>')
article=article.replace('?','?<eos>')
sentences=article.split('<eos>')

In [ ]:
sentences[3]

' In recent years, NASA instruments have painted a new picture of these strange objects that are, to many, the most fascinating objects in space.'

In [ ]:
max_chunk=300
current_chunk=0
chunks=[]

for sentence in sentences:
  if len(chunks)==current_chunk+1:
    if len(chunks[current_chunk])+len(sentence.split(' '))<=max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
      current_chunk +=1
      chunks.append(sentence.split(' '))
  else:
    print(current_chunk)
    chunks.append(sentence.split(' '))



0


In [ ]:
chunks[0]

['Black',
 "HolesDon't",
 'let',
 'the',
 'name',
 'fool',
 'you:',
 'a',
 'black',
 'hole',
 'is',
 'anything',
 'but',
 'empty',
 'space.',
 '',
 'Rather,',
 'it',
 'is',
 'a',
 'great',
 'amount',
 'of',
 'matter',
 'packed',
 'into',
 'a',
 'very',
 'small',
 'area',
 '-',
 'think',
 'of',
 'a',
 'star',
 'ten',
 'times',
 'more',
 'massive',
 'than',
 'the',
 'Sun',
 'squeezed',
 'into',
 'a',
 'sphere',
 'approximately',
 'the',
 'diameter',
 'of',
 'New',
 'York',
 'City.',
 '',
 'The',
 'result',
 'is',
 'a',
 'gravitational',
 'field',
 'so',
 'strong',
 'that',
 'nothing,',
 'not',
 'even',
 'light,',
 'can',
 'escape.',
 '',
 'In',
 'recent',
 'years,',
 'NASA',
 'instruments',
 'have',
 'painted',
 'a',
 'new',
 'picture',
 'of',
 'these',
 'strange',
 'objects',
 'that',
 'are,',
 'to',
 'many,',
 'the',
 'most',
 'fascinating',
 'objects',
 'in',
 'space.',
 'Intense',
 'X-ray',
 'flares',
 'thought',
 'to',
 'be',
 'caused',
 'by',
 'a',
 'black',
 'hole',
 'devouring',


In [ ]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id]=' '.join(chunks[chunk_id])

In [ ]:
chunks[3]

' The star clusters then sink to the center of the galaxy, where the intermediate-mass black holes merge to form a supermassive black hole. Discover More Topics From NASAJames Webb Space TelescopePerseverance RoverParker Solar ProbeJunoNASA explores the unknown in air and space, innovates for the benefit of humanity, and inspires the world through discovery. '

In [ ]:
len(chunks[1].split(' '))

286

## 4.Summarize the text

In [ ]:
res=summurizer(chunks,  do_sample=False)

Your max_length is set to 142, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


In [ ]:
res

[{'summary_text': ' A black hole is a great amount of matter packed into a very small area - think of a star ten times more massive than the Sun squeezed into a sphere approximately the diameter of New York City . The result is a gravitational field so strong that nothing, not even light, can escape . In recent years, NASA instruments have painted a new picture of these strange objects .'},
 {'summary_text': ' Most black holes form from the remnants of a large star that dies in a supernova explosion . As the surface of the star nears an imaginary surface called the "event horizon," time on the star slows relative to the time kept by observers . When the surface reaches the event horizon, time stands still, and the star can collapse no more .'},
 {'summary_text': " Astronomers spot black holes when another star draws near enough for some of the matter surrounding it to be snared by the black hole's gravity, churning out x-rays in the process . Most stellar black holes, however, are very

In [ ]:
' '.join([summ['summary_text'] for summ in res])

' A black hole is a great amount of matter packed into a very small area - think of a star ten times more massive than the Sun squeezed into a sphere approximately the diameter of New York City . The result is a gravitational field so strong that nothing, not even light, can escape . In recent years, NASA instruments have painted a new picture of these strange objects .  Most black holes form from the remnants of a large star that dies in a supernova explosion . As the surface of the star nears an imaginary surface called the "event horizon," time on the star slows relative to the time kept by observers . When the surface reaches the event horizon, time stands still, and the star can collapse no more .  Astronomers spot black holes when another star draws near enough for some of the matter surrounding it to be snared by the black hole\'s gravity, churning out x-rays in the process . Most stellar black holes, however, are very difficult to detect . One possible mechanism for the formati

## 5.ouput to text file

In [ ]:
text=' '.join([summ['summary_text'] for summ in res])

In [ ]:
with open('blogsummary.txt','w') as f:
  f.write(text)